In [1]:
%pwd

'/workspaces/End-End-Text-summeriser/research'

In [2]:
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
#model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail")

In [3]:
# change working directory to root directory
import os
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-End-Text-summeriser'

In [5]:
# update the config/config.yaml file for model training
# update the params.yaml file for model training. 
# Training parameters for the model. Preparing the model for training.

In [6]:
# 3. define the entity

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path # default path to the root directory
    data_path: Path # default path to the data directory
    model_ckpt: Path # default path to the model checkpoint directory
    num_train_epochs: int # default number of training epochs
    warmup_steps: int # default number of warmup steps
    per_device_train_batch_size: int # default batch size per device during training
    weight_decay: float # default weight decay coefficient
    logging_steps: int # default number of steps between logging training status
    evaluation_strategy: str # default evaluation strategy for evaluation
    eval_steps: int # default number of steps between evaluation
    save_steps: float # default number of steps between saving checkpoints
    gradient_accumulation_steps: int # default number of steps between gradient accumulation



In [8]:
# 4. configuration manager in src config.

In [9]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:

    #constructor assigning values from constant file to the class variables


    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # reading yaml config file and params file and assigning them to the class variables

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create directories/file for artifacts
        # note that we are using . to access, as we have defined using configbox

        create_directories([self.config.artifacts_root])

    #  get the model config 
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer # get the model config from the config file
        params = self.params.TrainingArguments # get the params from the params file

        create_directories([config.root_dir]) # create the root directory

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config # return the model config set in the config file



In [11]:

#5. components

In [12]:
# import library for model training
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/opt/conda/envs/newtexts/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-19 21:01:45,572: INFO: config: PyTorch version 2.2.1 available.]


In [13]:


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig): # initialize the model with the given config
        self.config = config


    # train the model
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu" # select the device to train on gpu or cpu
        print("device: ", device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt) # initialize the tokenizer with the model checkpoint
        print("tokenizer: completed")


        print("self.config.model_ckpt",self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device) # initialize the model with the model checkpoint
        print("model_pegasus loaded")

        
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus) # initialize the data collator with the tokenizer and model checkpoint for the batches
        print("seq2seq_data_collator completed ")
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        print("dataset_samsum_pt completed ")

        print("output_dir=", self.config.root_dir,
            "num_train_epochs=", type(self.config.num_train_epochs), 
            "warmup_steps=", type(self.config.warmup_steps),
            "per_device_train_batch_size=",  type(self.config.per_device_train_batch_size), 
            "per_device_eval_batch_size=", type(self.config.per_device_train_batch_size),
            "weight_decay=",type(self.config.weight_decay), 
            "logging_steps=",type(self.config.logging_steps),
            "evaluation_strategy=",type(self.config.evaluation_strategy), 
            "eval_steps=",type(self.config.eval_steps), 
            "save_steps=",type(self.config.save_steps), 
            "gradient_accumulation_steps=", type(self.config.gradient_accumulation_steps))

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=self.config.per_device_train_batch_size, 
            weight_decay=self.config.weight_decay, 
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, 
            eval_steps=self.config.eval_steps, 
            save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) 

     
        
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])

          
        print("training started: ")
        trainer.train()
        print("training completed: ")

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        print("model saved")
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        print("tokenizer saved")

In [14]:
# 6. pipeline

In [15]:
# pip install --upgrade accelerate 28 torch 2.2.1
# pip uninstall -y transformers accelerate
# pip install transformers accelerate
# pip install transformers[torch] accelerate

In [16]:
config = ConfigurationManager()

[2024-03-19 21:01:45,865: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-19 21:01:45,868: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-19 21:01:45,869: INFO: common: created directory at: artifacts]


In [17]:
model_trainer_config = config.get_model_trainer_config()

[2024-03-19 21:01:45,911: INFO: common: created directory at: artifacts/model_trainer]


In [18]:
model_trainer_config = ModelTrainer(config=model_trainer_config)

In [19]:
model_trainer_config.train()

device:  cpu
tokenizer: completed
self.config.model_ckpt google/pegasus-cnn_dailymail


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model_pegasus loaded
seq2seq_data_collator completed 
dataset_samsum_pt completed 
output_dir= artifacts/model_trainer num_train_epochs= <class 'int'> warmup_steps= <class 'int'> per_device_train_batch_size= <class 'int'> per_device_eval_batch_size= <class 'int'> weight_decay= <class 'float'> logging_steps= <class 'int'> evaluation_strategy= <class 'str'> eval_steps= <class 'int'> save_steps= <class 'str'> gradient_accumulation_steps= <class 'int'>
training started: 


/opt/conda/envs/newtexts/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


training completed: 
model saved
tokenizer saved


In [20]:
try:
    config = ConfigurationManager() # initialize the configuration manager
    model_trainer_config = config.get_model_trainer_config() # get the model trainer config from the config file
    model_trainer_config = ModelTrainer(config=model_trainer_config) # initialize the model trainer with the model trainer config
    model_trainer_config.train() # train the model
except Exception as e:
    raise e

[2024-03-19 21:15:52,227: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-19 21:15:52,229: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-19 21:15:52,230: INFO: common: created directory at: artifacts]
[2024-03-19 21:15:52,230: INFO: common: created directory at: artifacts/model_trainer]
device:  cpu
tokenizer: completed
self.config.model_ckpt google/pegasus-cnn_dailymail


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model_pegasus loaded
seq2seq_data_collator completed 
dataset_samsum_pt completed 
output_dir= artifacts/model_trainer num_train_epochs= <class 'int'> warmup_steps= <class 'int'> per_device_train_batch_size= <class 'int'> per_device_eval_batch_size= <class 'int'> weight_decay= <class 'float'> logging_steps= <class 'int'> evaluation_strategy= <class 'str'> eval_steps= <class 'int'> save_steps= <class 'str'> gradient_accumulation_steps= <class 'int'>
training started: 


/opt/conda/envs/newtexts/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


training completed: 
model saved
tokenizer saved


In [ ]:
# model/transformer from hugging face
model_ckpt = "google/pegasus-cnn_dailymail"

# create tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
print("model_ckpt:",model_ckpt)
print("tokenizer:",tokenizer)

print("model loading")

# model loading into device
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
     trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) 

    trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, 
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, 
            eval_steps=self.config.eval_steps, 
            save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) 
           trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 